In [11]:
import pandas as pd

# --------------------------
# 1) Resume Parser
# --------------------------
def parse_resume(row):
    """Extract structured resume fields directly from dataset row."""
    parsed = {
        "Resume_ID": row["Resume_ID"],
        "Name": row["Name"],
        "Skills": [s.strip() for s in row["Skills"].split(",")],
        "Experience(Years)": row["Experience (Years)"],
        "Education": row["Education"],
        "Certifications": row["Certifications"],
        "Job role": row["Job Role"], # Corrected column name
        "Recruiter Decision": row["Recruiter Decision"],
        "Salary Expectations": row["Salary Expectation ($)"],
        "Projects count": row["Projects Count"],
        "AI Score": row["AI Score (0-100)"]
    }
    return parsed

# --------------------------
# 2) Prompt Designer
# --------------------------
def rewrite_resume(parsed, job_keywords):
    """Rewrite skills and summary for job optimization."""

    summary = (
        f"{parsed['Name']} is applying for {parsed['Job role']} "
        f"with {parsed['Experience(Years)']} years of experience and "
        f"{parsed['Projects count']} completed projects. Skilled in {', '.join(parsed['Skills'][:4])}."
    )

    # Add job-specific keywords to skills
    all_skills = set(parsed["Skills"] + job_keywords)

    rewritten = {
        "Resume_ID": parsed["Resume_ID"],
        "Name": parsed["Name"],
        "Optimized Summary": summary,
        "Optimized Skills": list(all_skills),
        "Experience(Years)": parsed["Experience(Years)"],
        "Education": parsed["Education"],
        "Certifications": parsed["Certifications"],
        "Projects count": parsed["Projects count"]
    }
    return rewritten

# --------------------------
# 3) Evaluator
# --------------------------
def evaluate_resume(parsed, rewritten, job_keywords):
    """Evaluate resume fit for the job."""

    text_blob = " ".join(rewritten["Optimized Skills"] + [rewritten["Optimized Summary"]])

    # Keyword coverage
    coverage = sum(1 for kw in job_keywords if kw.lower() in text_blob.lower())
    coverage_score = round((coverage / len(job_keywords)) * 10, 1)

    # Consider AI score, recruiter decision, and project count
    ai_score = parsed["AI Score"]
    recruiter = parsed["Recruiter Decision"]
    projects = parsed["Projects count"]

    if coverage_score >= 7 and ai_score > 70 and recruiter == "Yes" and projects >= 3:
        final_eval = "Strong Resume"
    else:
        final_eval = "Needs Work"

    return {
        "Keyword Coverage Score": coverage_score,
        "AI Score": ai_score,
        "Recruiter Decision": recruiter,
        "Projects count": projects,
        "Final Evaluation": final_eval
    }

# --------------------------
# 4) Full Pipeline
# --------------------------
def process_dataset(csv_path, job_keywords, output_path="optimized_resumes.csv"):
    df = pd.read_csv(csv_path)
    results = []

    for idx, row in df.iterrows():
        parsed = parse_resume(row)
        rewritten = rewrite_resume(parsed, job_keywords)
        evaluation = evaluate_resume(parsed, rewritten, job_keywords)

        results.append({
            "Resume_ID": parsed["Resume_ID"],
            "Name": parsed["Name"],
            "Job role": parsed["Job role"],
            "Optimized Summary": rewritten["Optimized Summary"],
            "Optimized Skills": ", ".join(rewritten["Optimized Skills"]),
            "Experience(Years)": rewritten["Experience(Years)"],
            "Education": rewritten["Education"],
            "Certifications": rewritten["Certifications"],
            "Projects count": rewritten["Projects count"],
            "Keyword Coverage Score": evaluation["Keyword Coverage Score"],
            "AI Score": evaluation["AI Score"],
            "Recruiter Decision": evaluation["Recruiter Decision"],
            "Final Evaluation": evaluation["Final Evaluation"],
            "Salary Expectations": parsed["Salary Expectations"]
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)
    return results_df

# --------------------------
# Example Usage
# --------------------------
if __name__ == "__main__":
    # Example job keywords for "Data Analyst"
    job_keywords = ["SQL", "Python", "pandas", "Excel", "Tableau",
                    "ETL", "dashboard", "KPI", "reporting", "stakeholder"]

    dataset_path = "/content/AI_Resume_Screening.csv"   # <-- your dataset file
    results = process_dataset(dataset_path, job_keywords, output_path="optimized_resumes.csv")

    print("\n--- RESULTS PREVIEW ---")
    print(results.head())


--- RESULTS PREVIEW ---
   Resume_ID              Name               Job role  \
0          1        Ashley Ali          AI Researcher   
1          2      Wesley Roman         Data Scientist   
2          3     Corey Sanchez  Cybersecurity Analyst   
3          4  Elizabeth Carney          AI Researcher   
4          5        Julie Hill      Software Engineer   

                                   Optimized Summary  \
0  Ashley Ali is applying for AI Researcher with ...   
1  Wesley Roman is applying for Data Scientist wi...   
2  Corey Sanchez is applying for Cybersecurity An...   
3  Elizabeth Carney is applying for AI Researcher...   
4  Julie Hill is applying for Software Engineer w...   

                                    Optimized Skills  Experience(Years)  \
0  stakeholder, ETL, Python, TensorFlow, NLP, pan...                 10   
1  stakeholder, ETL, pandas, Python, Excel, repor...                 10   
2  stakeholder, ETL, Python, pandas, Excel, repor...                  

In [10]:
import pandas as pd

dataset_path = "/content/AI_Resume_Screening.csv"
df = pd.read_csv(dataset_path)
print(df.columns)

Index(['Resume_ID', 'Name', 'Skills', 'Experience (Years)', 'Education',
       'Certifications', 'Job Role', 'Recruiter Decision',
       'Salary Expectation ($)', 'Projects Count', 'AI Score (0-100)'],
      dtype='object')
